In [1]:
import pandas as pd
data_x = pd.read_csv('final_x.csv')  
data_y = pd.read_csv('before_standard_y.csv')  
# data_info = pd.read_csv('x_names_final.csv')

In [2]:
import numpy as np
columns = list(data_x.columns)
data_x_numpy = data_x.to_numpy()
data_y_numpy = data_y.to_numpy()

In [3]:
delete_index_x = []
delete_index_y = []

for i in range(len(columns)):
    if 'Unnamed' in data_x.columns[i]:
        delete_index_x.append(i)
    elif data_x.columns[i] == 'id':
        delete_index_x.append(i)

delete_index_x.reverse()    
for j in delete_index_x:
    data_x_numpy = np.delete(data_x_numpy, j, 1)
    del columns[j]
    
for m in range(len(list(data_y.columns))):
    if 'Unnamed' in data_y.columns[m]:
        delete_index_y.append(m)

delete_index_y.reverse()    
for n in delete_index_y:
    data_y_numpy = np.delete(data_y_numpy, n, 1)

In [4]:
data_x_numpy.shape, data_y_numpy.shape, len(columns)

((131624, 3967), (131624, 2), 3967)

## Feature Selection
使用Validation Threshold，threshold選0.9

In [5]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.95 * (1 - .95)))
X = sel.fit_transform(data_x_numpy)

In [6]:
X.shape

(131624, 85)

In [7]:
selected_column = []
for i in range(len(sel.get_support())):
    if sel.get_support()[i] == True:
        selected_column.append(columns[i])

In [8]:
selected_column

['appearDate',
 'enviornmentPic',
 'driverLicense',
 'other',
 'welfare',
 'jobDescription',
 'businessTrip',
 'workPeriod',
 'edu_不拘',
 'edu_大學以上',
 'edu_專科、大學',
 'edu_專科以上',
 'edu_高中、專科、大學',
 'edu_高中以上',
 'language_英文',
 'specialty_Excel',
 'specialty_Outlook',
 'specialty_PowerPoint',
 'specialty_Word',
 'jobCategory_作業員／包裝員',
 'jobCategory_國內業務人員',
 'jobCategory_店長／賣場管理人員',
 'jobCategory_門市／店員／專櫃人員',
 'jobCategory_餐廚助手',
 'jobCategory_餐飲服務生',
 'adressArea_台中市',
 'adressArea_台北市',
 'adressArea_台南市',
 'adressArea_新北市',
 'adressArea_桃園市',
 'adressArea_高雄市',
 'vacationPolicy_依公司規定',
 'vacationPolicy_週休二日',
 'startWorkingDay_一個月內',
 'startWorkingDay_一週內',
 'startWorkingDay_不限',
 'startWorkingDay_兩週內',
 'industry_餐館業',
 'apply_male_ratio',
 'apply_major_一般商業學類',
 'apply_major_不拘',
 'apply_major_企業管理相關',
 'apply_major_其他相關科系',
 'apply_major_國際貿易相關',
 'apply_major_普通科',
 'apply_major_英美語文相關',
 'apply_major_觀光事務相關',
 'apply_major_資訊管理相關',
 'apply_major_電機電子工程相關',
 'apply_major_餐旅服務相關',
 'ap

## Split Train, Valid, Stack and Test

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, data_y_numpy, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)

In [10]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_valid shape:', X_valid.shape)
print('y_valid shape:', y_valid.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)

X_train shape: (84239, 85)
y_train shape: (84239, 2)
X_valid shape: (21060, 85)
y_valid shape: (21060, 2)
X_test shape: (26325, 85)
y_test shape: (26325, 2)


## Linear Regression

In [11]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression
reg = MultiOutputRegressor(LinearRegression()).fit(X_train, y_train)

In [12]:
ypred = reg.predict(X_test)

reg_rmse = []
reg_rmse.append(np.mean(np.square(y_test[:, 0] - ypred[:, 0])) ** 0.5)
reg_rmse.append(np.mean(np.square(y_test[:, 1] - ypred[:, 1])) ** 0.5)

reg_mae = []
reg_mae.append(np.mean(np.abs(y_test[:, 0] - ypred[:, 0])))
reg_mae.append(np.mean(np.abs(y_test[:, 1] - ypred[:, 1])))

print('test rmse:', reg_rmse)
print('test mae: ', reg_mae)

test rmse: [5613.547082338637, 19538.92398005586]
test mae:  [3548.8520263087657, 10733.732848780046]


## Lasso
tune alpha(regularization strength)
有Convergence Warning，將max iter設為1000，並關閉warning

In [13]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn import linear_model
import warnings
warnings.filterwarnings('ignore')

lasso_alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
lasso_rmse = np.zeros((len(lasso_alpha), 2))
lasso_mae = np.zeros((len(lasso_alpha), 2))

for i in range(len(lasso_alpha)):
    lasso = MultiOutputRegressor(linear_model.Lasso(alpha=lasso_alpha[i], max_iter=1000)).fit(X_train, y_train)
    ypred = lasso.predict(X_valid)
    lasso_rmse[i, 0] = np.mean(np.square(y_valid[:, 0] - ypred[:, 0])) ** 0.5
    lasso_rmse[i, 1] = np.mean(np.square(y_valid[:, 1] - ypred[:, 1])) ** 0.5
    lasso_mae[i, 0] = np.mean(np.abs(y_valid[:, 0] - ypred[:, 0]))
    lasso_mae[i, 1] = np.mean(np.abs(y_valid[:, 1] - ypred[:, 1]))

In [14]:
lasso_rmse, lasso_mae

(array([[ 6089.33188843, 21106.83349379],
        [ 6089.33145855, 21106.83176349],
        [ 6089.32782307, 21106.81440217],
        [ 6089.34264048, 21106.62529834],
        [ 6088.41239581, 21105.72143785],
        [ 6094.20903842, 21113.67553911],
        [ 6238.05579529, 21219.86881462]]),
 array([[ 3604.06618653, 11090.79403766],
        [ 3604.06446654, 11090.78965246],
        [ 3604.0480512 , 11090.74520599],
        [ 3603.93661802, 11090.25325932],
        [ 3602.50563652, 11085.78637886],
        [ 3603.95405312, 11066.18618119],
        [ 3748.5413322 , 11013.31323067]]))

In [15]:
lasso_index = np.argmin(np.sum(lasso_rmse, axis=1))
best_lasso_alpha = lasso_alpha[lasso_index]

In [16]:
best_lasso_alpha

1

選rmse加總最小者

In [17]:
best_lasso = MultiOutputRegressor(linear_model.Lasso(alpha=best_lasso_alpha)).fit(X_train, y_train)
ypred = best_lasso.predict(X_test)

best_lasso_rmse = []
best_lasso_rmse.append(np.mean(np.square(y_test[:, 0] - ypred[:, 0])) ** 0.5)
best_lasso_rmse.append(np.mean(np.square(y_test[:, 1] - ypred[:, 1])) ** 0.5)

best_lasso_mae = []
best_lasso_mae.append(np.mean(np.abs(y_test[:, 0] - ypred[:, 0])))
best_lasso_mae.append(np.mean(np.abs(y_test[:, 1] - ypred[:, 1])))

In [18]:
best_lasso_rmse, best_lasso_mae

([5614.205021779923, 19537.00259773125],
 [3547.2685713795727, 10728.808247156756])

## Ridge
tune alpha(regularization strength)

In [19]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn import linear_model

ridge_rmse = np.zeros((len(lasso_alpha), 2))
ridge_mae = np.zeros((len(lasso_alpha), 2))
ridge_alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]

for i in range(len(ridge_alpha)):
    ridge = linear_model.Ridge(alpha=i).fit(X_train, y_train)
    ypred = ridge.predict(X_valid)
    ridge_rmse[i, 0] = np.mean(np.square(y_valid[:, 0] - ypred[:, 0])) ** 0.5
    ridge_rmse[i, 1] = np.mean(np.square(y_valid[:, 1] - ypred[:, 1])) ** 0.5
    ridge_mae[i, 0] = np.mean(np.abs(y_valid[:, 0] - ypred[:, 0]))
    ridge_mae[i, 1] = np.mean(np.abs(y_valid[:, 1] - ypred[:, 1]))

In [20]:
ridge_rmse, ridge_mae

(array([[ 6088.79507863, 21105.3131988 ],
        [ 6088.9795625 , 21105.16857616],
        [ 6089.10590965, 21105.18777591],
        [ 6089.180739  , 21105.23551035],
        [ 6089.22499328, 21105.28654571],
        [ 6089.25058217, 21105.33480609],
        [ 6089.2642079 , 21105.37900322]]),
 array([[ 3604.04196042, 11088.92917427],
        [ 3603.95179893, 11087.69378382],
        [ 3603.92585523, 11087.12139163],
        [ 3603.95497968, 11086.81240051],
        [ 3603.96081395, 11086.56619453],
        [ 3603.95541969, 11086.34571775],
        [ 3603.94435221, 11086.20988625]]))

In [21]:
ridge_index = np.argmin(np.sum(ridge_rmse, axis=1))
best_ridge_alpha = ridge_alpha[ridge_index]

In [22]:
best_ridge_alpha

0.0001

In [23]:
best_ridge = MultiOutputRegressor(linear_model.Ridge(alpha=best_ridge_alpha)).fit(X_train, y_train)
ypred = best_ridge.predict(X_test)

best_ridge_rmse = []
best_ridge_rmse.append(np.mean(np.square(y_test[:, 0] - ypred[:, 0])) ** 0.5)
best_ridge_rmse.append(np.mean(np.square(y_test[:, 1] - ypred[:, 1])) ** 0.5)

best_ridge_mae = []
best_ridge_mae.append(np.mean(np.abs(y_test[:, 0] - ypred[:, 0])))
best_ridge_mae.append(np.mean(np.abs(y_test[:, 1] - ypred[:, 1])))

In [24]:
best_ridge_rmse, best_ridge_mae

([5613.547106719652, 19538.923872374613], [3548.852009104005, 10733.732616861])

## Random Forest
tune max depth, n_estimators

In [25]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.ensemble import RandomForestRegressor

param_grid = {'estimator__n_estimators':np.arange(5,1000,10),
              'estimator__max_depth': [5, 10, 20]}

model = RSCV(MultiOutputRegressor(RandomForestRegressor()), param_grid, n_iter = 15).fit(X_train, y_train)
best_random_forest = model.best_estimator_

In [26]:
ypred = best_random_forest.predict(X_test)
best_forest_rmse = []
best_forest_rmse.append(np.mean(np.square(y_test[:, 0] - ypred[:, 0])) ** 0.5)
best_forest_rmse.append(np.mean(np.square(y_test[:, 1] - ypred[:, 1])) ** 0.5)
best_forest_mae = []
best_forest_mae.append(np.mean(np.abs(y_test[:, 0] - ypred[:, 0])))
best_forest_mae.append(np.mean(np.abs(y_test[:, 1] - ypred[:, 1])))

In [27]:
best_forest_rmse, best_forest_mae

([5041.300810321682, 15018.332595331487],
 [3144.2036233210297, 8090.188764325724])

In [28]:
model.best_estimator_

MultiOutputRegressor(estimator=RandomForestRegressor(max_depth=20,
                                                     n_estimators=465))